# Homework 1 Breast Cancer

The data in this homework is from http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29

You can read what each column means from the website.

1) (0)First write a function that separate the data into 20% test and 80% train. Hint: use np.random.seed to fix random number generator.

- What could happen if you just pick last 20%?
- Why do we need to randomize?

In [1]:
import numpy as np
from glob import glob

def readFile(file):
    tmp = []
    with open(file) as f:
        for line in f:
            tmp.append(line)
    return tmp
def cleanData(data):
    return [d.rstrip('\n').split(',')[1:] for d in data]
         
def toTwo(data):
    test = []
    train = []
#     np.random.seed(100)
    rd = [np.random.randint(10) for x in xrange(len(data))]
    for i in xrange(len(data)):
        if rd[i] <=1:
            test.append(data[i])
        else:
            train.append(data[i])
    return test,train

def toFloat(data):
    tmp = []
    for dd in data:
        tp = []
        add = True
        for d in dd:
            if d == '?':
                add = False
                break
            tp.append(float(d))
        if add:
            tmp.append(tp[::])
    return tmp
            
dataRaw = readFile("breast-cancer-wisconsin.data.txt")
data = cleanData(dataRaw)
data = toFloat(data)
test, train = toTwo(data)

print len(test), len(train), len(data)
print test[0]

137 546 683
[2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 5.0, 2.0]


2) (0) There are many features in the data. Tell me one from the data set that you should NOT use.

id number

3) (100) Build a Bayes Classifier for figuring out whether the tumor is cancerour or benign. Train on 80% and test on the 20%.
- Justify the prior you use.
- How many do you get correctly on test dataset?
- Do you think the the number we got correctly on the test dataset is the thing we should optimize? Why and why not? (Exercise your common sense)

In [2]:
import math
def fourOrTwo(data):
    four, two = [], []
    for d in data:
        if d[9] == 2: 
            two.append(d[:9])
        else:
            four.append(d[:9])
    return four, two

four, two = fourOrTwo(train)

In [3]:
def featuresCount(data):
    f = [0]*10
    fc = []
    idxs = xrange(9)
    
    for i in idxs:
        fc.append(f[::])
    
    for d in data:
        for i in idxs:
            fc[i][int(d[i] - 1)] += 1
    return fc

def cal_fprob(fc):
    f = [0]*10
    fp = []
    idxs = xrange(9)
    jdxs = xrange(10)
    for i in idxs:
        fp.append(f[::])  
    
    for i in idxs:
        s = sum(fc[i])
        for j in jdxs:
            fp[i][j] = fc[i][j]/float(s)
            
    return fp

def testData(data):
    fcF = featuresCount(four)
    fcT = featuresCount(two)
    fpF = cal_fprob(fcF)
    fpT = cal_fprob(fcT)
    
    idxs = xrange(9)
    c = 0
    
    priorT = len(four)/ (len(four) + len(two) + 0.0)
    priorF = len(two)/ (len(four) + len(two) + 0.0)
    
    for d in data:
        isT = 1
        isF = 1
        for i in idxs:
            isT *= fpT[i][int(d[i] - 1)] if fpT[i][int(d[i] - 1)] != 0 else 1
            isF *= fpF[i][int(d[i] - 1)] if fpF[i][int(d[i] - 1)] != 0 else 1 

        nFactor = isT*priorT + isF*priorF
        isT = isT*priorT/nFactor
        isF = isF*priorF/nFactor

        if (d[9] == 4):
            if isF > isT:
                c += 1
        else:
            if isF < isT:
                c += 1
    return c

print testData(test)/float(len(test))

0.970802919708


4) (Bonus) Try to do something different from what you did in 3) using your original idea. Report what you did even if it is worse than what you did in 3. If possible report why it works better/worse.

Do not lookup the paper/other method just yet. I want you to develop the gut feeling in data analysis.

Grading will be on scale of (meh 1%, OK? 3%, Hmm interesting 5%, Wow 10%)



In [11]:
def featuresCount(data):
    f = [0]*10
    fc = []
    idxs = xrange(9)
    
    for i in idxs:
        fc.append(f[::])
    
    for d in data:
        for i in idxs:
            fc[i][int(d[i] - 1)] += 1
    return fc

def cal_fprob(fc):
    f = [0]*10
    fp = []
    idxs = xrange(9)
    jdxs = xrange(10)
    for i in idxs:
        fp.append(f[::])  
    
    for i in idxs:
        s = sum(fc[i])
        for j in jdxs:
            fp[i][j] = fc[i][j]/float(s)
            
    return fp

def testData(data):
    fcF = featuresCount(four)
    fcT = featuresCount(two)
    fpF = cal_fprob(fcF)
    fpT = cal_fprob(fcT)
    
    idxs = xrange(9)
    c = 0
    
    priorT = len(four)/ (len(four) + len(two) + 0.0)
    priorF = len(two)/ (len(four) + len(two) + 0.0)
    
    for d in data:
        isT = 1
        isF = 1
        for i in idxs:
            isT *= fpT[i][int(d[i] - 1)] if fpT[i][int(d[i] - 1)] != 0 else 1
            isF *= fpF[i][int(d[i] - 1)] if fpF[i][int(d[i] - 1)] != 0 else 1 

        nFactor = isT*priorT + isF*priorF
        isT = isT*priorT/nFactor

        if (d[9] == 4):
            if isT < 0.5:
                c += 1
        else:
            if isT >= 0.5:
                c += 1
    return c

print testData(test)/float(len(test))

0.970802919708
